In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
try:
    identity_df = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
    transaction_df = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
except FileNotFoundError:
    print("Error: One or both of the CSV files were not found. Please make sure the file paths are correct.")
    #  To prevent the rest of the code from running and causing errors.
    exit()

In [3]:
%pip install dagshub 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install mlflow 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import mlflow
import mlflow.sklearn
import dagshub

try:
    # Initialize Dagshub only if the repo info is correct
    dagshub.init(repo_owner='konstantine25b', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)
    print("DagsHub initialized successfully.")
    mlflow.set_experiment("IEEE-CIS Fraud Detection_PreProcessing")
    print(f"MLflow experiment set to: {mlflow.get_experiment_by_name('IEEE-CIS Fraud Detection_PreProcessing').name}")
except Exception as e:
    print(f"Could not initialize DagsHub or set MLflow experiment: {e}")
    print("Proceeding without MLflow tracking.")
    # Set a dummy client to avoid errors if tracking fails
    mlflow_active = False
else:
    mlflow_active = True

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=f9188c79-5e8e-4a04-ba1b-b3f5b65033cf&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6ff684995d60ccbb3ccb1dda0d0dd920c939c518b4e0a38656b7387a2c50eeb9




Output()

Could not initialize DagsHub or set MLflow experiment: Expecting value: line 1 column 1 (char 0)
Proceeding without MLflow tracking.


# Explore Identity Table

In [6]:
print("Identity Table Exploration:")
print(f"Number of rows: {len(identity_df)}")
print(f"Shape of the DataFrame: {identity_df.shape}")
print("\nMissing values per column:")
print(identity_df.isnull().sum().sort_values(ascending=False))


Identity Table Exploration:
Number of rows: 144233
Shape of the DataFrame: (144233, 41)

Missing values per column:
id_24            139486
id_25            139101
id_07            139078
id_08            139078
id_21            139074
id_26            139070
id_23            139064
id_27            139064
id_22            139064
id_18             99120
id_03             77909
id_04             77909
id_33             70944
id_09             69307
id_10             69307
id_30             66668
id_32             66647
id_34             66428
id_14             64189
DeviceInfo        25567
id_13             16913
id_16             14893
id_06              7368
id_05              7368
id_20              4972
id_19              4915
id_17              4864
id_31              3951
DeviceType         3423
id_02              3361
id_28              3255
id_29              3255
id_11              3255
id_15              3248
id_35              3248
id_36              3248
id_37              3

In [7]:
identity_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [8]:
# Calculate the percentage of missing values
missing_percentage = (identity_df.isnull().sum() / len(identity_df)) * 100
missing_percentage = missing_percentage.sort_values(ascending=False)

In [9]:
# Display columns with more than 20% missing values
print("\nColumns with more than 20% missing values in Identity Table:")
cols_to_print = missing_percentage[missing_percentage > 0]
if len(cols_to_print) > 0:
  print(cols_to_print)
else:
  print("No columns with more than 10% missing values")


Columns with more than 20% missing values in Identity Table:
id_24         96.708798
id_25         96.441868
id_07         96.425922
id_08         96.425922
id_21         96.423149
id_26         96.420375
id_23         96.416215
id_27         96.416215
id_22         96.416215
id_18         68.722137
id_03         54.016071
id_04         54.016071
id_33         49.187079
id_09         48.052110
id_10         48.052110
id_30         46.222432
id_32         46.207872
id_34         46.056034
id_14         44.503685
DeviceInfo    17.726179
id_13         11.726165
id_16         10.325654
id_06          5.108401
id_05          5.108401
id_20          3.447200
id_19          3.407681
id_17          3.372321
id_31          2.739318
DeviceType     2.373243
id_02          2.330257
id_28          2.256765
id_29          2.256765
id_11          2.256765
id_15          2.251912
id_35          2.251912
id_36          2.251912
id_37          2.251912
id_38          2.251912
dtype: float64


# Explore Transaction Table

In [10]:
print("\nTransaction Table Exploration:")
print(f"Number of rows: {len(transaction_df)}")
print(f"Shape of the DataFrame: {transaction_df.shape}")
print("\nMissing values per column:")
print(transaction_df.isnull().sum().sort_values(ascending=False))


Transaction Table Exploration:
Number of rows: 590540
Shape of the DataFrame: (590540, 394)

Missing values per column:
dist2            552913
D7               551623
D13              528588
D14              528353
D12              525823
                  ...  
C1                    0
C2                    0
C14                   0
isFraud               0
TransactionID         0
Length: 394, dtype: int64


In [11]:
# Calculate the percentage of missing values
missing_percentage = (transaction_df.isnull().sum() / len(transaction_df)) * 100
missing_percentage = missing_percentage.sort_values(ascending=False)

In [12]:
# Display columns with more than 60% missing values
print("\nColumns with more than 60% missing values in Transaction Table:")
cols_to_print = missing_percentage[missing_percentage > 60]
if len(cols_to_print) > 0:
  print(cols_to_print)
else:
  print("No columns with more than 60% missing values")


Columns with more than 60% missing values in Transaction Table:
dist2    93.628374
D7       93.409930
D13      89.509263
D14      89.469469
D12      89.041047
           ...    
V239     76.053104
V238     76.053104
V234     76.053104
V227     76.053104
V222     76.053104
Length: 168, dtype: float64


In [13]:
# --- Check for TransactionID uniqueness ---
print("\nTransactionID Uniqueness Check:")
print(f"Number of unique TransactionIDs in identity_df: {identity_df['TransactionID'].nunique()}")
print(f"Number of unique TransactionIDs in transaction_df: {transaction_df['TransactionID'].nunique()}")
print(f"Number of rows in identity_df: {len(identity_df)}")
print(f"Number of rows in transaction_df: {len(transaction_df)}")



TransactionID Uniqueness Check:
Number of unique TransactionIDs in identity_df: 144233
Number of unique TransactionIDs in transaction_df: 590540
Number of rows in identity_df: 144233
Number of rows in transaction_df: 590540


In [14]:
# Check if all TransactionIDs in identity_df are in transaction_df
identity_ids = set(identity_df['TransactionID'])
transaction_ids = set(transaction_df['TransactionID'])
not_in_transaction = identity_ids - transaction_ids
not_in_identity = transaction_ids - identity_ids
if len(not_in_transaction) > 0:
    print(f"\nNumber of TransactionIDs in identity_df but not in transaction_df: {len(not_in_transaction)}")
    if (len(not_in_transaction) < 20):
        print(f"Example IDs: {list(not_in_transaction)[:5]}")  # Print only a few if there are many
else:
    print("\nAll TransactionIDs in identity_df are present in transaction_df")

if len(not_in_identity) > 0:
    print(f"Number of TransactionIDs in transaction_df but not in identity_df: {len(not_in_identity)}")
    if (len(not_in_identity) < 20):
        print(f"Example IDs: {list(not_in_identity)[:5]}")
else:
    print("All TransactionIDs in transaction_df are present in identity_df")


All TransactionIDs in identity_df are present in transaction_df
Number of TransactionIDs in transaction_df but not in identity_df: 446307


# Do Cleaning

### Filter High NaNs Separately

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import WOEEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import time
import os


In [16]:
if mlflow_active:
    mlflow.start_run(run_name=f"preprocessing_pipeline_{time.strftime('%Y%m%d_%H%M%S')}")
    print("MLflow run started.")

In [17]:
class HighNARemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold
        self.columns_to_drop = None
        
    def fit(self, X, y=None):
        # Calculate percentage of missing values for each column
        na_percentage = X.isna().mean()
        # Identify columns with missing values above threshold
        self.columns_to_drop = na_percentage[na_percentage > self.threshold].index.tolist()
        return self
    
    def transform(self, X):
        # Drop columns with high missing values
        return X.drop(columns=self.columns_to_drop, errors='ignore')
    
    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return []
        return [f for f in input_features if f not in self.columns_to_drop]

In [18]:
# First, remove high NA columns from both datasets
identity_na_remover = HighNARemover(threshold=0.2)
transaction_na_remover = HighNARemover(threshold=0.6)

# Fit and transform to remove high NA columns
identity_df_filtered = identity_na_remover.fit_transform(identity_df)
transaction_df_filtered = transaction_na_remover.fit_transform(transaction_df)

In [19]:
print("\n--- After Removing High NA Columns ---")
print(f"Identity filtered shape: {identity_df_filtered.shape}")
print(f"Transaction filtered shape: {transaction_df_filtered.shape}")
print(f"Columns dropped from identity: {len(identity_na_remover.columns_to_drop)}")
print(f"Columns dropped from transaction: {len(transaction_na_remover.columns_to_drop)}")



--- After Removing High NA Columns ---
Identity filtered shape: (144233, 22)
Transaction filtered shape: (590540, 226)
Columns dropped from identity: 19
Columns dropped from transaction: 168


In [20]:
# Log NA removal metrics
if mlflow_active:
    mlflow.log_param("identity_na_threshold", 0.2)
    mlflow.log_param("transaction_na_threshold", 0.6)
    mlflow.log_param("identity_cols_dropped", len(identity_na_remover.columns_to_drop))
    mlflow.log_param("transaction_cols_dropped", len(transaction_na_remover.columns_to_drop))
    mlflow.log_param("identity_cols_remaining", identity_df_filtered.shape[1])
    mlflow.log_param("transaction_cols_remaining", transaction_df_filtered.shape[1])
    
    # Log the names of dropped columns
    mlflow.log_text("\n".join(identity_na_remover.columns_to_drop), "identity_dropped_columns.txt")
    mlflow.log_text("\n".join(transaction_na_remover.columns_to_drop), "transaction_dropped_columns.txt")


In [21]:
# Now split the transaction data
X_transaction = transaction_df_filtered.drop('isFraud', axis=1)
y_transaction = transaction_df_filtered['isFraud']

X_transaction_train, X_transaction_test, y_train, y_test = train_test_split(
    X_transaction, y_transaction, test_size=0.2, random_state=42, stratify=y_transaction
)

In [22]:
# Get the corresponding identity records for train and test sets
train_transaction_ids = X_transaction_train['TransactionID'].values
test_transaction_ids = X_transaction_test['TransactionID'].values

# Filter identity dataframe based on TransactionIDs
X_identity_train = identity_df_filtered[identity_df_filtered['TransactionID'].isin(train_transaction_ids)]
X_identity_test = identity_df_filtered[identity_df_filtered['TransactionID'].isin(test_transaction_ids)]


In [23]:
# Create a mapping from TransactionID to target for identity data
transaction_id_to_target = dict(zip(transaction_df_filtered['TransactionID'], transaction_df_filtered['isFraud']))
y_identity_train = X_identity_train['TransactionID'].map(transaction_id_to_target)
y_identity_test = X_identity_test['TransactionID'].map(transaction_id_to_target)


In [24]:
print("\n--- Train-Test Split Information ---")
print(f"Transaction train set shape: {X_transaction_train.shape}")
print(f"Transaction test set shape: {X_transaction_test.shape}")
print(f"Identity train set shape: {X_identity_train.shape}")
print(f"Identity test shape: {X_identity_test.shape}")
print(f"Transaction target train distribution: \n{y_train.value_counts(normalize=True)}")
print(f"Identity target train distribution: \n{y_identity_train.value_counts(normalize=True)}")



--- Train-Test Split Information ---
Transaction train set shape: (472432, 225)
Transaction test set shape: (118108, 225)
Identity train set shape: (115658, 22)
Identity test shape: (28575, 22)
Transaction target train distribution: 
isFraud
0    0.965011
1    0.034989
Name: proportion, dtype: float64
Identity target train distribution: 
TransactionID
0    0.921908
1    0.078092
Name: proportion, dtype: float64


In [25]:
# Log train-test split metrics
if mlflow_active:
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("transaction_train_rows", X_transaction_train.shape[0])
    mlflow.log_param("transaction_test_rows", X_transaction_test.shape[0])
    mlflow.log_param("identity_train_rows", X_identity_train.shape[0])
    mlflow.log_param("identity_test_rows", X_identity_test.shape[0])
    mlflow.log_param("train_fraud_rate", float(y_train.mean()))
    mlflow.log_param("test_fraud_rate", float(y_test.mean()))


In [26]:
# Function to categorize columns
def categorize_columns(df):
    # Identify numeric and categorical columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # For categorical columns, separate those with ≤4 unique values and >4 unique values
    categorical_low_cardinality = []
    categorical_high_cardinality = []
    
    for col in categorical_cols:
        if df[col].nunique() <= 4:
            categorical_low_cardinality.append(col)
        else:
            categorical_high_cardinality.append(col)
    
    # Remove 'TransactionID' from numeric columns if present
    if 'TransactionID' in numeric_cols:
        numeric_cols.remove('TransactionID')
    
    return numeric_cols, categorical_low_cardinality, categorical_high_cardinality


In [27]:
# Now categorize columns
identity_numeric_cols, identity_cat_low, identity_cat_high = categorize_columns(X_identity_train)
transaction_numeric_cols, transaction_cat_low, transaction_cat_high = categorize_columns(X_transaction_train)


In [28]:
print("\n--- Column Categorization ---")
print(f"Identity numeric columns: {len(identity_numeric_cols)}")
print(f"Identity low cardinality categorical columns: {len(identity_cat_low)}")
print(f"Identity high cardinality categorical columns: {len(identity_cat_high)}")
print(f"Transaction numeric columns: {len(transaction_numeric_cols)}")
print(f"Transaction low cardinality categorical columns: {len(transaction_cat_low)}")
print(f"Transaction high cardinality categorical columns: {len(transaction_cat_high)}")



--- Column Categorization ---
Identity numeric columns: 9
Identity low cardinality categorical columns: 10
Identity high cardinality categorical columns: 2
Transaction numeric columns: 211
Transaction low cardinality categorical columns: 11
Transaction high cardinality categorical columns: 2


In [29]:
if mlflow_active:
    mlflow.log_param("identity_numeric_cols", len(identity_numeric_cols))
    mlflow.log_param("identity_cat_low_cols", len(identity_cat_low))
    mlflow.log_param("identity_cat_high_cols", len(identity_cat_high))
    mlflow.log_param("transaction_numeric_cols", len(transaction_numeric_cols))
    mlflow.log_param("transaction_cat_low_cols", len(transaction_cat_low))
    mlflow.log_param("transaction_cat_high_cols", len(transaction_cat_high))
    
    # Log column names by category
    mlflow.log_text("\n".join(identity_numeric_cols), "identity_numeric_columns.txt")
    mlflow.log_text("\n".join(identity_cat_low), "identity_cat_low_columns.txt")
    mlflow.log_text("\n".join(identity_cat_high), "identity_cat_high_columns.txt")
    mlflow.log_text("\n".join(transaction_numeric_cols), "transaction_numeric_columns.txt")
    mlflow.log_text("\n".join(transaction_cat_low), "transaction_cat_low_columns.txt")
    mlflow.log_text("\n".join(transaction_cat_high), "transaction_cat_high_columns.txt")


In [30]:
# Create identity preprocessing pipeline
identity_preprocessor = Pipeline([
    ('column_transformer', ColumnTransformer([
        ('num_imputer', Pipeline([
            ('imputer', SimpleImputer(strategy='mean'))
        ]), identity_numeric_cols),
        ('cat_low_encoder', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), identity_cat_low),
        ('cat_high_encoder', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('woe', WOEEncoder(handle_missing='return_nan'))
        ]), identity_cat_high)
    ], remainder='drop'))
])

In [31]:
# Create transaction preprocessing pipeline
transaction_preprocessor = Pipeline([
    ('column_transformer', ColumnTransformer([
        ('num_imputer', Pipeline([
            ('imputer', SimpleImputer(strategy='mean'))
        ]), transaction_numeric_cols),
        ('cat_low_encoder', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), transaction_cat_low),
        ('cat_high_encoder', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('woe', WOEEncoder(handle_missing='return_nan'))
        ]), transaction_cat_high)
    ], remainder='drop'))
])


In [32]:
if mlflow_active:
    mlflow.log_param("numeric_imputation_strategy", "mean")
    mlflow.log_param("categorical_imputation_strategy", "most_frequent")
    mlflow.log_param("low_cardinality_encoding", "one-hot")
    mlflow.log_param("high_cardinality_encoding", "weight-of-evidence")


In [33]:
# Fit and transform the identity data
start_time = time.time()
identity_preprocessor.fit(X_identity_train, y_identity_train)
X_identity_train_processed = identity_preprocessor.transform(X_identity_train)
X_identity_test_processed = identity_preprocessor.transform(X_identity_test)
identity_processing_time = time.time() - start_time

In [34]:
# Fit and transform the transaction data
start_time = time.time()
transaction_preprocessor.fit(X_transaction_train, y_train)
X_transaction_train_processed = transaction_preprocessor.transform(X_transaction_train)
X_transaction_test_processed = transaction_preprocessor.transform(X_transaction_test)
transaction_processing_time = time.time() - start_time

In [35]:
# After transformation, print more detailed information
print("\n--- Processed Data Information ---")
print(f"Processed identity train data shape: {X_identity_train_processed.shape}")
print(f"Processed identity test data shape: {X_identity_test_processed.shape}")
print(f"Processed transaction train data shape: {X_transaction_train_processed.shape}")
print(f"Processed transaction test data shape: {X_transaction_test_processed.shape}")
print(f"Identity processing time: {identity_processing_time:.2f} seconds")
print(f"Transaction processing time: {transaction_processing_time:.2f} seconds")



--- Processed Data Information ---
Processed identity train data shape: (115658, 32)
Processed identity test data shape: (28575, 32)
Processed transaction train data shape: (472432, 240)
Processed transaction test data shape: (118108, 240)
Identity processing time: 1.21 seconds
Transaction processing time: 13.38 seconds


In [36]:
if mlflow_active:
    mlflow.log_metric("identity_processing_time", identity_processing_time)
    mlflow.log_metric("transaction_processing_time", transaction_processing_time)
    mlflow.log_param("identity_processed_features", X_identity_train_processed.shape[1])
    mlflow.log_param("transaction_processed_features", X_transaction_train_processed.shape[1])


In [37]:
print("\n--- Checking for Remaining NaN Values ---")
if isinstance(X_identity_train_processed, np.ndarray):
    identity_nan_count = np.isnan(X_identity_train_processed).sum()
    transaction_nan_count = np.isnan(X_transaction_train_processed).sum()
    print(f"Identity train NaN count: {identity_nan_count}")
    print(f"Transaction train NaN count: {transaction_nan_count}")
else:
    identity_nan_count = X_identity_train_processed.isna().sum().sum()
    transaction_nan_count = X_transaction_train_processed.isna().sum().sum()
    print(f"Identity train NaN count: {identity_nan_count}")
    print(f"Transaction train NaN count: {transaction_nan_count}")



--- Checking for Remaining NaN Values ---
Identity train NaN count: 0
Transaction train NaN count: 0


In [38]:
if mlflow_active:
    mlflow.log_metric("identity_remaining_nan_count", float(identity_nan_count))
    mlflow.log_metric("transaction_remaining_nan_count", float(transaction_nan_count))


In [39]:
class IdentityPipelineWrapper(Pipeline):
    def predict(self, X):
        """Add a predict method that just returns the transformed data.
        This allows MLflow to log the model with python_function flavor."""
        return self.transform(X)

class TransactionPipelineWrapper(Pipeline):
    def predict(self, X):
        """Add a predict method that just returns the transformed data.
        This allows MLflow to log the model with python_function flavor."""
        return self.transform(X)

In [40]:
# Create wrapped versions of our pipelines
identity_pipeline_wrapped = IdentityPipelineWrapper(
    steps=[
        ('na_remover', identity_na_remover),
        ('preprocessor', identity_preprocessor)
    ]
)

transaction_pipeline_wrapped = TransactionPipelineWrapper(
    steps=[
        ('na_remover', transaction_na_remover),
        ('preprocessor', transaction_preprocessor)
    ]
)


In [41]:
# Create proper input examples for model signatures with some missing values
identity_example = X_identity_train.iloc[:5].copy()
# Introduce some NaN values to ensure proper schema inference
for col in identity_example.select_dtypes(include=['int64']).columns[:2]:
    identity_example.loc[0, col] = np.nan

transaction_example = X_transaction_train.iloc[:5].copy()
# Introduce some NaN values to ensure proper schema inference
for col in transaction_example.select_dtypes(include=['int64']).columns[:2]:
    transaction_example.loc[0, col] = np.nan

In [42]:
# Log the wrapped models with input examples
if mlflow_active:
    # Log identity pipeline with signature
    mlflow.sklearn.log_model(
        identity_pipeline_wrapped, 
        "identity_pipeline_model",
        input_example=identity_example
    )
    
    # Log transaction pipeline with signature
    mlflow.sklearn.log_model(
        transaction_pipeline_wrapped, 
        "transaction_pipeline_model",
        input_example=transaction_example
    )
    
    # Log sample data and statistics as before
    mlflow.log_text(str(X_identity_train.iloc[0].to_dict()), "identity_sample_input.json")
    mlflow.log_text(str(X_transaction_train.iloc[0].to_dict()), "transaction_sample_input.json")
    mlflow.log_text(pd.DataFrame(X_identity_train_processed).describe().to_string(), "identity_processed_stats.txt")
    mlflow.log_text(pd.DataFrame(X_transaction_train_processed).describe().to_string(), "transaction_processed_stats.txt")
    
    # End the MLflow run
    mlflow.end_run()
    print("MLflow run completed and artifacts logged.")